# Глава 23. Пакеты модулей

Помимо возможности импортировать имя модуля существует возможность импортировать имена каталогов. Каталог на языке Python называется **пакетом**, поэтому такая операция импортирования называется **импортированием пакетов**.

Операция импортирования пакета превращает имя каталога в еще одну разновидность пространства имен, в котором атрибутам соответствуют подкаталоги и файлы модулей, находящиеся в этих каталогах.

## Основы операции импортирования пакетов

В инструкциях `import` и `from` там, где указывалось имя простого файла, можно указать список имен в пути к каталогу, разделяя их символами точки:

```
import dir1.dir2.mod
```

```
from dir1.dir2.mod import x
```

Эти инструкции предполагают, что каталог `dir1` находится внутри некоторого контейнерного каталога `dir0`, который находится в пути поиска модулей.

Если `dir0` нет в пути поиска модулей, его необходимо туда добавить

### Пакеты и настройка пути поиска

Если `dir0` - имя каталога, которое требуется добавить в путь поиска модулей, то для того, чтобы импортировать модуль, находящийся в `/home/dir0/dir1/dir2/mod` нужно добавить путь `/home/dir0` в переменную окружения `PYTHONPATH` или в файл `.pth` и использовать инструкцию `import dir1.dir2.mod`

### Файлы `__init__.py` пакетов

При импортировании пакетов существует еще одно условие, которое необходимо соблюдать: каждый каталог в пути, указанном в инструкции импортирования пакета, должен содержать файл с именем `__init__.py`, в противном случае операция импорта пакета будет терпеть неудачу.

Т.е. в примере выше каталоги `dir1` и `dir2` должны содержать этот файл, каталог `dir0` может не содержать такой файл, потому что сам он  не указан в инструкции импортирования пакета.

Структура каталогов в примере должна иметь следующий вид:

```
dir0/   # каталог-контейнер в пути поиска модулей
    dir1/
        __init__.py
        dir2/
            __init__.py
            mod.py

```

Файлы `__init__.py` могут содержать программный код на языке Python, как любые другие файлы модулей. Отчасти они являются объявлениями для интерпретатора и могут вообще ничего не содержать.

>Эти файлы, будучи объявлениями, предотвращают неумышленное сокрытие в каталогах с совпадающими именами истинно требуемых модулей, если они отображаются позже в списке поиска модулей. Без этого защитного механизма интерпретатор мог бы выбирать каталоги, которые не имеют никакого отношения к вашему программному коду, только лишь потому, что в пути поиска они появляются ранее.

>Символ точки как разделитель имен каталогов был выбран не только для обеспечения независимости от используемой платформы, но и потому, что пути в инструкциях `import` в действительности становятся вложенными объектами пути. Этот синтаксис также подразумевает, что вы будете получать невразумительные сообщения об ошибках, если забудете опустить расширение `.py`. Например, инструкция `import mod.py` подразумевает, что выполняется импорт пути к каталогу, – она загрузит `mod.py`, затем попытается загрузить `mod/py.py`, что в конечном счете приведет к появлению сбивающего с толку сообщения об ошибке `No module named py` (Модуль с именем `py` не найден).

В общем случае **файл `__init__.py` предназначен для выполнения действий по инициализации пакета, создание пространства имен для каталога и реализации поведения инструкций `from ... import *`, когда они используются для импортирования каталогов**:

* **Инициализация пакета** Когда интерпретатор импортирует каталог в первый раз, он автоматически запускает код файла `__init__.py` этого каталога. По этой причине эти файлы обычно содержат код, выполняющий действия по инициализации, необходимые для файлов в пакете. Например, он может использоваться для создания файлов с данными, открытия соединения с базой данных и т.д.. Обычно файлы `__init__.py` не предназначены для непосредственного выполнения - они запускаются автоматически, когда выполняется первое обращение к пакету.


* **Инициализация пространства имен модуля**. При импортировании пакетов пути к каталогам в вашем сценарии после завершения операции импортирования превращаются в настоящие иерархии вложенных объектов. В предыдущем примере после завершения операции импорта можно будет использовать выражение `dir1.dir2`, которое возвращает объект модуля, чье пространство содержит все имена, определяемые файлом `__init__.py` из каталога `dir2`. Такие файлы создают пространства имен для объектов модулей, соответствующих каталогам, в которых отсутствуют настоящие файлы модулей.


* **Поведение инструкции `from *`**. В качестве дополнительной особенности, в файлах `__init__.py` можно использовать **списки `__all__`**, чтобы определить, что будет импортироваться из каталога инструкцией `from *`. Список `__all__` представляет собой список имен субмодулей, которые должны импортироваться, когда в инструкции `from *` указывается имя пакета (каталога). Если списка нет, инструкция `from *` не будет автоматически загружать субмодули, вложенные в каталог, - она загрузит только имена, определяемые инструкциями присваивания в файле `__init__.py`, включая любые субмодули, явно импортируемые программным кодом в этом файле.

## Пример импортирования пакета

Следующие три файла располагаются в каталоге `dir1` и в подкаталоге `dir2` – комментарии описывают пути к этим файлам:
```
# Файл: dir1/__init__.py
print('dir1 init')
x = 1
```

```
# Файл: dir1/dir2/__init__.py
print('dir2 init')
y = 2
```

```
# Файл: dir1/dir2/mod.py
print('in mod.py')
z = 3
```

Для каталога, вмещающего `dir1` (домашнего или любого, включенного в путь поиска модулей), не требуется наличие файла `__init__.py`

Инструкции `import` выполняют файлы инициализации в **каждом каталоге**, которые присутствуют в пути к модулю.

Кроме того, как и файлы модулей, уже импортированные каталоги могут передаваться функции `reload` для принудительного повторного исполнения этого единственного элемента.

In [1]:
import sys
sys.path.append('./exercises/5_23/')

In [2]:
import dir1.dir2.mod

dir1 init
dir2 init
in mod.py


In [3]:
import dir1.dir2.mod  # повторное импортирование не выполняется

In [4]:
from imp import reload

reload(dir1)

dir1 init


<module 'dir1' from './exercises/5_23/dir1/__init__.py'>

In [5]:
reload(dir1.dir2)

dir2 init


<module 'dir1.dir2' from './exercises/5_23/dir1/dir2/__init__.py'>

>После операции импортирования путь, указанный в инструкции `import`, становится **цепочкой вложенных объектов**

In [6]:
dir1

<module 'dir1' from './exercises/5_23/dir1/__init__.py'>

In [7]:
dir1.dir2

<module 'dir1.dir2' from './exercises/5_23/dir1/dir2/__init__.py'>

In [8]:
dir1.dir2.mod

<module 'dir1.dir2.mod' from './exercises/5_23/dir1/dir2/mod.py'>

Каждый каталог в пути фактически становится переменной, которой присваивается объект модуля, пространство имен которого инициализируется всеми инструкциями присваивания в  файле `__init__.py`, находящемся в  этом каталоге.

Имя `dir1.x` ссылается на переменную `x`, которой присваивается значение в файле `dir1/__init__.py`, точно так же, как имя `mod.z` ссылается на переменную `z`, которой присваивается значение в файле mod.py

In [9]:
dir1.x

1

In [10]:
dir1.dir2.mod.z

3

### Инструкции `from` и `import` для пакетов

Если попытаться непосредственно обратиться к `dir2` или `mod`, будет получено сообщение об ошибке:

In [11]:
dir2.mod

NameError: name 'dir2' is not defined

Поэтому для импортирования пакетов часто более удобно использовать инструкцию `from`, чтобы избежать необходимости ввода полного имени при каждом обращении к нему.

>Еще более важно следующее: если вы когда-нибудь произведете реструктуризацию дерева каталогов, то в  случае использования инструкции `from` достаточно будет обновить путь только в самой этой инструкции, тогда как в случае использования инструкции `import` придется обновлять все обращения к именам в изменившемся пакете.

In [12]:
from dir1.dir2 import mod  # описание пути находится только в этом месте

In [13]:
mod.z  # указывать полный путь не требуется

3

## Когда используется операция импортирования пакетов?

* Прежде всего, так как операция импортирования пакетов содержит некоторые сведения о  структуре каталогов, где находятся файлы программы, они, в  первую очередь, упрощают поиск файлов и  служат организационным инструментом.


* Если вы организовали размещение своих файлов в дереве каталогов по функциональным признакам, то операция импортирования пакетов делает более очевидной роль, которую играют пакеты, что обеспечивает более высокую удобочитаемость программного кода. (`import utilities` vs `import database.client.utilities`)


* Операция импортирования пакетов может также упростить задание переменной окружения PYTHONPATH и  файлов .pth, хранящих настройки пути поиска модулей.


* Операция импортирования пакетов способна разрешать неоднозначности за счет явного и точного указания импортируемых файлов

### История о трех программах

Программа 1:

```
system1/
       utilities.py # Общие вспомогательные функции, классы
       main.py      # Этот файл запускает программу
       other.py     # Импортирует и использует модуль utilities
```

Программа 2:

```
system2/
       utilities.py # Общие вспомогательные функции, классы
       main.py      # Этот файл запускает программу
       other.py     # Импортирует и использует модуль utilities
```

Предположим, что после установки этих двух программ вы решили
использовать вспомогательные функции из обоих файлов `utilities.py` в  своей собственной программе.

В  такой ситуации вам необходима возможность из своего программного кода, хранящегося в  третьем каталоге, загрузить один из двух файлов:
```
import utilities
utilities.func('spam')
```

Теперь проблема начинает вырисовываться. Чтобы вообще выполнить эту работу, вам придется включить в путь поиска модулей каталоги, содержащие файлы `utilities.py`. Но какой каталог поместить первым – `system1` или `system2`?


Проблема заключается в линейной природе пути поиска. Он всегда просматривается слева направо. Независимо от того, как долго вы будете ломать голову над этой проблемой, вы всегда будете получать файл `utilities.py` из каталога, который находится в пути поиска раньше (левее). Как следствие, вы никогда не сможете импортировать одноименный файл из другого каталога.

Эту проблему можно решить с помощью пакетов. Вместо того, чтобы устанавливать программы как плоские списки файлов в независимые каталоги, можно установить их в подкаталоги с общим корнем.

Например, можно было бы организовать установку всего программного кода из этого примера в виде следующей иерархии:

```
root/
    system1/
        __init__.py
        utilities.py
        main.py
        other.py
    system2/
        __init__.py
        utilities.py
        main.py
        other.py
    system3/         # Здесь или в другом месте
        __init__.py  # располагается ваш новый программный код
        myfile.py
```

Теперь достаточно просто добавить общий корневой каталог в путь поиска модулей. Если выполнять импортирование относительно этого общего корня, можно будет с помощью операции импортирования пакетов импортировать
любой файл из любой программы – использование имени вмещающего каталога делает путь (а, значит, и ссылку на модуль) уникальным.

Фактически можно даже импортировать обе утилиты сразу в одном и том же модуле, при условии, что вы будете использовать инструкцию `import` и при каждом обращении к именам будете указывать полный путь к вспомогательным модулям:
```
import system1.utilities
import system2.utilities
system1.utilities.function('spam')
system2.utilities.function('eggs')
```

Обратите внимание, что операции импорта в  обеих оригинальных программах продолжают действовать без изменений.

**Поскольку в  этих программах поиск модулей производится в первую очередь в их домашних каталогах, добавление общего корневого каталога в путь поиска модулей никак не отражается на программном коде в `system1` и `system2`** – они по-прежнему могут использовать инструкции `import utilities` и получать в ответ свои собственные файлы.

## Импортирование относительно пакета

### Изменения в Python 3.0

В Python 3.0 в операцию импортирования внутри пакетов было внесено два изменения:

* Изменилась семантика пути поиска модулей так, что теперь операция импортирования модуля по умолчанию пропускает собственный каталог пакета. Она проверяет только компоненты пути поиска. Эта операция называется **импортированием по «абсолютному» пути**.


* Расширен синтаксис инструкции `from` так, что теперь имеется возможность явно указать, что поиск импортируемых модулей должен производиться только в  каталоге пакета. Эта операция называется **импортированием по «относительному» пути**.

### Основы импортирования по относительному пути

В инструкции `from` теперь могут использовать точки (`.`), чтобы указать, что поиск модулей в  первую очередь должен производиться в том же самом пакете (эта особенность известна как **импортирование относительно пакета**), а  не где-то в  другом месте, то есть:

* в инструкции `from` в начале пути можно использовать точки, чтобы указать, что импорт должен производиться **относительно** вмещающего пакета, - при таком способе импортирования поиск модулей будет производиться только внутри пакета, а модули с теми же именами, находящиеся где-то в пути поиска (`sys.path`), будут недоступны. Благодаря этому модули внутри пакета получают преимущество перед модулями за его пределами.

* В Python 3 по умолчанию выполняется импортирование по абсолютному пути - при отсутствии точек операции импортирования пропускают вмещающий пакет и пытаются отыскать импортируемые модули в пути поиска `sys.path`

Например, `from . import spam` предписывает интерпретатору импортировать модуль с именем `spam`, расположенный в том же пакете, что и файл, где находится эта инструкция.

Аналогично, следующая инструкция `from .spam import name` означает: из модуля с именем `spam`, расположенного в том же пакете, что и файл, где находится эта инструкция, импортировать переменную `name`.

>Обратите внимание: **ведущий символ точки может использоваться только в инструкции `from`** – в инструкции `import` он недопустим.

Допустим, что имеется каталог `mypkg` пакета, тогда следующие альтернативные варианты импортирования внутри этого пакета будут работать так, как описывается:

```
from .string import name1, name2   # Импорт имен из mypkg.string
from . import string               # Импорт mypkg.string
from .. import string  # Импорт string из родительского каталога
```

### Зачем необходим импорт относительно пакета?

Эта возможность предназначена, чтобы дать сценариям возможность ликвидировать возникающие неоднозначности, которые могут возникать, когда в разных местах в пути поиска присутствует несколько одноименных модулей.

### Решение проблемы с импортированием относительно пакета в 3.0

С целью разрешить эту дилемму поведение операции импортирования внутри
пакетов в Python 3.0 (и в виде дополнительной возможности в 2.6) было изменено так, что теперь она выполняет импорт только по абсолютному пути.

```
import string
# Импортирует модуль string за пределами пакета
```

```
from string import name
# Импортирует имя name из модуля string за пределами пакета
```

```
from . import string
# Импортирует mypkg.string (относительно пакета)
```

```
from .string import name1, name2 # Импортирует имена из mypkg.string
```

По сути, символ точки `.` в инструкции относительного импорта представляет каталог в пакете, содержащий файл, где выполняется операция импортирования. **Дополнительная начальная точка предписывает выполнить относительный импорт, начиная с  родительского каталога текущего пакета**.

Например, инструкция:
```
from .. import spam
# Импортирует модуль одного уровня с пакетом mypkg
```
загрузит модуль, находящийся на том же уровне в иерехии каталогов, что и пакет `mypkg`, – то есть модуль `spam`, находящийся в каталоге, родительском по отношению к пакету `mypkg`.

В общем случае действия программного кода в модуле **`A.B.C`** будут следующими:
```
from . import D    # Импортирует A.B.D (. означает A.B)
from .. import E   # Импортирует A.E (.. означает A)
from .D import X   # Импортирует A.B.D.X (. означает A.B)
from ..E import X  # Импортирует A.E.X (.. означает A)
```

### Правила импортирования по относительному пути

* **Операция импортирования по относительному пути применяется исключительно для импортирования внутри пакета**


* **Импортирование по относительному пути возможно только с помощью инструкции `from`**


* **Неоднозначность терминологии**. В действительности, все операции импортирования выполняются относительно чего-нибудь. За пределами пакетов импортирование выполняется относительно каталогов, перечисленных в пути поиска модулей `sys.path`. В 3.0 обычная инструкция импортирования по "абсолютному" пути пропускает каталог пакета (и импортирование выполняется относительно каталогов, перечисленных в `sys.path`), а инструкции импорта со "специальным" синтаксисом импортирования относительно текущего пакета выполняют поиск только в пределах пакета.

### Правила поиска модулей

* Для простых имен пакетов (например, `A`) поиск выполняется во всех каталогах, перечисленных в списке `sys.path`, слева направо. Этот список конструируется из системных значений по умолчанию и из настроек пользователя.


* Пакеты - это обычные каталоги с модулями на языке Python, содержащие специальный файл `__init__.py`, который позволяет указывать в инструкциях импортирования цепочки каталогов вида `A.B.C`. Чтобы получить возможность импортировать, например, `A.B.C`, каталог `A` должен находиться в одном из каталогов, перечисленных в пути поиска модулей `sys.path`, `B` должен быть подкаталогом пакета в каталоге `A`, а `C` должен быть модулем или другим компонентом в каталоге `B`, доступным для импортирования.


* Внутри файлов пакета обычные инструкции `import` выполняют поиск модулей в `sys.path` в соответствии с теми же правилами, что и инструкции импортирования в любых других модулях. Однако при импортировании с использованием инструкций `from` и начальных точек в именах поиск выполняется относительно текущего пакета, то есть поиск производится только в каталоге текущего пакета, а обычный поиск в `sys.path` не выполняется.

### Относительный импорт в примерах

#### Импортирование за пределами пакетов

Следующий пример загрузит модуль `string` из стандартной библиотеки, как и предполагалось:

```
>>> import string
```

Но если в текущий рабочий каталог добавить модуль с тем же именем, будет загружен он, вместо библиотечного модуля, потому что текущий рабочий каталог стоит на первом месте в пути поиска:

```
>>> import string
>>> string
<module 'string' from 'string.py'>
```

**Синтаксис импортирования относительно текущего пакета считается недопустимым для использования в файлах, не входящих в состав пакета**:

```
>>> from . import string
Tracebak (most recent call last):
  File "<stdin>", line 1, in <module>
ImportError: cannot import name 'string'
```

#### Импортирование внутри пакетов

```
# Иерархия примера

test/pkg/
        __init__.py
        spam.py
        eggs.py
```

```
# test/pkg/spam.py
import eggs
print(eggs.X)
```

```
# test/pkg/eggs.py
X = 99999
import string
print(string)
```

Запускаем Python в каталоге `test`:

In [14]:
import sys
sys.path.append('./exercises/5_23/test/')

In [15]:
import pkg.spam

ModuleNotFoundError: No module named 'eggs'

Получаем ошибку, т.к. в файле `spam.py` необходимо использовать специальный синтаксис операции импортирования относительно текущего пакета: **`from . import eggs`**

```
# test/pkg/spam_corrected.py
from . import eggs
print(eggs.X)
```

```
# test/pkg/eggs.py
X = 99999
import string
print(string)
```

**Импорт по-прежнему выполняется относительно текущего рабочего каталога**

**Модули по-прежнему могут импортировать модули из стандартной библиотеки** (выше в примере `import string` в `eggs.py`). **Но если в домашнем каталоге (не в пакете) лежит модуль с таким же именем, то будет импортирован он**.

```
# test/string.py
print('string' * 8)

# test/pkg/spam_corrected.py
from . import eggs
print(eggs.X)

# test/pkg/eggs.py
X = 99999
import string  # <== Импортирует модуль string из текущего рабочего
print(string)  # каталога, а не из стандартной библиотеки!
```

Запускаем Python в каталоге `test`:

In [16]:
!cd ./exercises/5_23/test/ && python -c 'import pkg.spam_corrected'

stringstringstringstringstringstringstringstring
<module 'string' from '/home/shurik/Desktop/Lutz_Learning_Python/exercises/5_23/test/string.py'>
99999


#### Выбор модулей операциями импортирования по относительному и абсолютному пути

Если создать модуль `string.py` внутри пакета `pkg`:

```
# test/pkg/spam.py
import string
print(string)

# test/pkg/string.py
print('Ni' * 8)
```

То в данном случае в Python 3 импорт `spam` импортирует модуль `string`  из стандартной библиотеки.

Если же в файле `spam.py` изменить импорт на относительный:

```
# test/pkg/spam.py
from . import string
```

То при импорте `spam` будет импортирован модуль `string` из пакета

#### Импорт по-прежнему выполняется относительно текущего рабочего каталога (еще раз)

Инструкции импортирования по абсолютному пути позволяют пропускать модули пакета, однако они по-прежнему зависят от других элементов списка `sys.path`. В последнем нашем примере попробуем определить два собственных модуля `string`. Один модуль с этим именем находится в текущем рабочем каталоге, другой – в пакете и еще один – в стандартной библиотеке:

```
# test\string.py
print('string' * 8)

# test\pkg\spam.py
from . import string
print(string)

# test\pkg\string.py
print('Ni' * 8)
```

Когда мы импортируем модуль `string` с применением синтаксиса импортирования относительно пакета, мы получаем версию модуля из пакета, как и следовало ожидать:

```
C:\test> c:\Python30\python
>>> import pkg.spam
NiNiNiNiNiNiNiNi
<module ‘pkg.string’ from ‘pkg\string.py’>
```

Однако, когда используется синтаксис импортирования по абсолютному пути Python 3.0 рассматривает ее, как «абсолютную», то есть просто пропускает каталог пакета и  импортирует модуль из текущего рабочего каталога (не из стандартной библиотеки):

```
# test\string.py
print('string' * 8)

# test\pkg\spam.py
import string
print(string)

# test\pkg\string.py
print('Ni' * 8)


C:\test> c:\Python30\python
>>> import pkg.spam
stringstringstringstringstringstringstringstring
<module ‘string’ from ‘string.py’>
```

В данном случае для файла программы, использующей пакет, модуль в стандартной библиотеке оказывается недоступным.

Поскольку разрешающая способность операции импортирования зависит от окружения, которое невозможно предсказать заранее, операция импортирования по абсолютному пути в версии 3.0 не гарантирует, что модуль будет найден в стандартной библиотеке.